In [1]:
"""
Class that implements a regression-cnn
"""
from ipywidgets import *
import tensorflow as tf
import vae_tests.tfutils as ut
import vae_tests.config as config
import numpy as np
import vae_tests.augmentation as aug


from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

@ops.RegisterGradient("MaxPoolWithArgmax")
def _MaxPoolWithArgmaxGrad(op, grad, some_other_arg):
  return gen_nn_ops._max_pool_grad(op.inputs[0],
                                   op.outputs[0],
                                   grad,
                                   op.get_attr("ksize"),
                                   op.get_attr("strides"),
                                   padding=op.get_attr("padding"),
                                   data_format='NHWC')


def loss(x1, x2):
    # Euclidean distance between x1,x2
    l2diff = tf.sqrt( tf.reduce_sum(tf.square(tf.sub(x1, x2)),
                                    reduction_indices=1))
    return l2diff

class RegressCNN():
    
    def __init__(self):
        # Training Parameters
        self.learning_rate   = 0.001 #0.001
        self.training_epochs = 1000 #1000
        self.batch_size      = 10
        self.size_output_pose = 2 #dimensions of the output
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.99)
        self.sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
        #self._buildGraph()
        self._buildGraphSimple()
        self.defCost()
        self.initTrain()
        self.loadData()
        
    def loadData(self):
        # read the training data
        path = config.path
        file = 'shelf'
        path_data = path + file + '.npz'
        l = np.load(path_data)
        print (l.files)

        # Parse data
        self.train_depth = l['depth_map']
        self.train_obj_pose = l['poses']
              
    
    
    def initTrain(self):
        # Saver 
        save_step = 10;
        saver = tf.train.Saver(max_to_keep=self.training_epochs) 

        #adam optimisation. Needs the cost function
        self.optm = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
        #init = tf.global_variables_initializer()
        init = tf.initialize_all_variables()        
        #sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
        
        self.sess.run(init, feed_dict={self.phase: True})
    
    def defCost(self):
        #loss =  tf.nn.l2_loss(self.pred, self.y_in)
        #self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(self.pred,self.y_in))
        self.cost = tf.reduce_mean(loss(self.pred,self.y_in))
        
    def _buildGraphSimple(self):
        
        self.x_in_depth = tf.placeholder(tf.float32, [None, 128, 128, 1])
        self.y_in = tf.placeholder(tf.float32, [None, 2])

        self.phase = tf.placeholder(tf.bool, name='phase_train')
        phase = self.phase
        self.conv1 = ut.conv_layer_with_bn(
                        self.x_in_depth, [3, 3, 1, 64], phase, name="conv1")
         # see https://groups.google.com/a/tensorflow.org/forum/#!topic/discuss/vf8eH9YMwVA
        dyn_input_shape = tf.shape(self.conv1)
        in_batch = dyn_input_shape[0]
        
        self.pool1, self.pool1_indices = tf.nn.max_pool_with_argmax(
                        self.conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME', name='pool1')        
        self.conv2 = ut.conv_layer_with_bn(
                        self.pool1, [3, 3, 64, 128], phase, name="conv2")
        self.pool2, self.pool2_indices = tf.nn.max_pool_with_argmax(
                        self.conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2') 
        
        self.conv3 = ut.conv_layer_with_bn(
                        self.pool2, [3, 3, 128, 256], phase, name="conv3")
        
        self.pool3, self.pool3_indices = tf.nn.max_pool_with_argmax(
                        self.conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool3')        
    
        
        self.dense = tf.reshape(self.pool3, [in_batch, self.pool3.get_shape().as_list()[1]*self.pool3.get_shape().as_list()[2]*self.pool3.get_shape().as_list()[3]])
        print('dense')
        print(self.dense.get_shape())
        self.full = ut.full_layer_with_bn(self.dense, [self.pool3.get_shape().as_list()[1]*self.pool3.get_shape().as_list()[2]*self.pool3.get_shape().as_list()[3], 256], phase, name="full")
        print('full')
        print(self.full.get_shape())
        self.pred = ut.full_layer_with_bn(self.full, [256, self.size_output_pose], phase, name="pred")
        print('pred')
        print(self.pred.get_shape())
    
    def _buildGraph(self):
        self.x_in_depth = tf.placeholder(tf.float32, [None, 128, 128, 1])
        self.y_in = tf.placeholder(tf.float32, [None, 2])

        self.phase = tf.placeholder(tf.bool, name='phase_train')
        phase = self.phase
        self.conv1 = ut.conv_layer_with_bn(
                        self.x_in_depth, [3, 3, 1, 64], phase, name="conv1")
         # see https://groups.google.com/a/tensorflow.org/forum/#!topic/discuss/vf8eH9YMwVA
        dyn_input_shape = tf.shape(self.conv1)
        in_batch = dyn_input_shape[0]
        
        self.pool1, self.pool1_indices = tf.nn.max_pool_with_argmax(
                        self.conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME', name='pool1')        
        self.conv2 = ut.conv_layer_with_bn(
                        self.pool1, [3, 3, 64, 128], phase, name="conv2")
        self.pool2, self.pool2_indices = tf.nn.max_pool_with_argmax(
                        self.conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')       
        self.conv3 = ut.conv_layer_with_bn(
                        self.pool2, [3, 3, 128, 256], phase, name="conv3")
        
        self.pool3, self.pool3_indices = tf.nn.max_pool_with_argmax(
                        self.conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool3')        
        self.conv4 = ut.conv_layer_with_bn(
                        self.pool3, [3, 3, 256, 512], phase, name="conv4")
       
        self.pool4, pool4_indices = tf.nn.max_pool_with_argmax(
                        self.conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool4')

        print('pool4')
        print(self.pool4.get_shape())
        self.dense = tf.reshape(self.pool4, [in_batch, self.pool4.get_shape().as_list()[1]*self.pool4.get_shape().as_list()[2]*512])
        print('dense')
        print(self.dense.get_shape())
        self.full = ut.full_layer_with_bn(self.dense, [self.pool4.get_shape().as_list()[1]*self.pool4.get_shape().as_list()[2]*512, 256], phase, name="full")
        print('full')
        print(self.full.get_shape())
        self.pred = ut.full_layer_with_bn(self.full, [256, self.size_output_pose], phase, name="pred")
        print('pred')
        print(self.pred.get_shape())
        
    def train(self):
        for epoch in range(self.training_epochs):
            avg_cost = 0.
            test_acc = 0.
            print('training...')


            #1st, data augmentation
            print('   augmenting for training...')
            train_depth_aug = np.empty([self.train_depth.shape[0], self.train_depth.shape[1], self.train_depth.shape[2], 1])
            for i in range(self.train_depth.shape[0]):               
                img_aug = self.train_depth[i].reshape((self.train_depth[i].shape[0], self.train_depth[i].shape[1], 1))
                train_depth_aug[i] = img_aug
            print('   done')
            print(train_depth_aug.shape)

            ntrain = train_depth_aug.shape[0]
            num_batch = int(ntrain/self.batch_size)+1


            for i in range(num_batch):         
                randidx = np.random.randint(ntrain, size=self.batch_size)            
                batch_depth = train_depth_aug[randidx, :]
                batch_y_ins = self.train_obj_pose[randidx, :]
                         
                
                #img=batch_depth[0,:]
                #img_deaug = img.reshape(img.shape[0], img.shape[1])
                #print(img_deaug.shape)
                #display_img_array(img_deaug)       
                feed={self.x_in_depth: batch_depth, self.y_in : batch_y_ins , self.phase:True}
                self.sess.run(self.optm, feed_dict=feed)
                # Compute average loss
                fetches = [self.cost, self.pred]
                local_cost, prediction  = self.sess.run(fetches, feed_dict=feed)
                
                avg_cost = 1.
                #avg_cost += local_cost/num_batch
                if i == num_batch - 1:
                    print('gt:')
                    print(batch_y_ins)
                    print('prediction:')
                    print(prediction)
                    print('cost:')
                    print(local_cost)


            #error_train.append(avg_cost)
            #loss.append(avg_cost)
            #print('avg_loss=',avg_cost)
            print ("Optimization Finished!")


In [8]:
#regress.train_depth.shape
a= [0.5 , 0.3]
b = [0.5, 0.2]

tf.reduce_mean(a)


<tf.Tensor 'Mean_2:0' shape=() dtype=float32>

In [2]:
regress = RegressCNN()
regress.train()


dense
(?, 65536)
full
(?, 256)
pred
(?, 2)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
['poses', 'depth_map']
training...
   augmenting for training...
   done
(290, 128, 128, 1)
gt:
[[ 0.69813156  0.        ]
 [ 0.43633222 -0.34906578]
 [ 0.          0.08726644]
 [-0.26179933  0.17453289]
 [-1.13446378  0.17453289]
 [-0.95993089  0.26179933]
 [-0.87266444  0.        ]
 [-0.26179933 -0.34906578]
 [ 0.34906578 -0.08726644]
 [-1.13446378 -0.43633222]]
prediction:
[[ 0.94552428 -0.37023351]
 [ 0.43618944 -0.51989543]
 [ 0.05079961  0.93854135]
 [-0.12351613  0.81426752]
 [-0.78326482  0.22074907]
 [-0.77452248  0.8488574 ]
 [-0.44606751 -0.74381125]
 [-0.34598401 -0.78553045]
 [ 0.91915095 -0.64921808]
 [-0.76972663 -0.54587913]]
cost:
0.557657
Optimization Finished!
training...
   augmenting for training...
   done
(290, 128, 128, 1)
gt:
[[ 0.34906578 -0.08726644]
 [-0.95993089 -0.43633222]
 [ 0.34906578 -0.08726644]
 [ 0.785398    0.26179933]
 [ 0.08726644 

In [ ]:
# %% We now create a new session to actually perform the initialization the
# variables:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# %% We'll train in minibatches and report accuracy:
n_epochs = 10
batch_size = 100
for epoch_i in range(n_epochs):
    for batch_i in range(mnist.train.num_examples // batch_size):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={
            x: batch_xs, y: batch_ys, is_training: True})
    print(sess.run(accuracy,
                   feed_dict={
                       x: mnist.validation.images,
                       y: mnist.validation.labels,
                       is_training: False
}))

NameError: name 'l' is not defined